## ２層ニューラルネットワーク

### 各種メソッドの定義

In [5]:
import numpy as np
import sys, os

sys.path.append("../deep-learning-from-scratch-master")

In [6]:
def sigmoid(x: np.ndarray | float):
    """スカラーにも任意次元の配列にも適用可能なシグモイド関数"""
    return 1 / (1 + np.exp(-x))


def cross_entropy_error(y, t):
    """予測値yと正解ラベルtの交差エントロピー誤差を返す。１次元のベクトルに対しても、行列（ミニバッチ）に対しても適用可。"""

    # バッチがない場合（yが１次元のベクトルの場合）も、1×n 行列に変換しておく
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換する
    if t.size == y.size:
        t = t.argmax(axis=1)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


def softmax(x):
    """ミニバッチに対しても適用できるsoftmax関数"""
    x = x - np.max(x, axis=-1, keepdims=True)  # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)


def numerical_gradient(f, x):
    """行列に対して勾配を計算する関数"""
    h = 1e-4
    grad = np.zeros_like(x)

    # 多次元配列xのインデックスをタプル形式で返すイテレータ
    it = np.nditer(x, flags=["multi_index"])

    while not it.finished:

        # 次のインデックスを取得
        idx = it.multi_index

        # x_idxの現在の値を保持
        tmp = x[idx]

        # x_idx + h のときの値を計算
        x[idx] = tmp + h
        fxh1 = f(x)

        # x_idx - h のときの値を計算
        x[idx] = tmp - h
        fxh2 = f(x)

        # x_idxに関する偏微分を計算
        grad[idx] = (fxh1 - fxh2) / (2 * h)

        # xの値を元に戻す
        x[idx] = tmp

        # 次のインデックス
        it.iternext()

    return grad

### `TwoLayerNet` クラスの定義

In [7]:
class TwoLayerNet:

    def __init__(
        self,
        input_size: int,
        hidden_size: int,
        output_size: int,
        weight_init_std: float = 0.01,
    ):
        """重みを初期化する"""
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(
            input_size, hidden_size
        )  # １層目の重みをガウス分布で初期化
        self.params["b1"] = np.zeros(hidden_size)  # １層目のバイアスを0で初期化
        self.params["W2"] = weight_init_std * np.random.randn(
            hidden_size, output_size
        )  # ２層目の重みをガウス分布で初期化
        self.params["b2"] = np.zeros(output_size)  # ２層目のバイアスを0で初期化

    def predict(self, x):
        """現在のパラメータに基づいて出力を返す"""
        a1 = np.dot(x, self.params["W1"]) + self.params["b1"]
        z1 = sigmoid(a1)
        a2 = np.dot(z1, self.params["W2"]) + self.params["b2"]
        y = softmax(a2)
        return y

    def loss(self, x, t):
        """交差エントロピー誤差を計算する。xは入力値、tは正解データ。"""
        y = self.predict(x)
        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        """推測精度を計算する。xは入力値、tは正解データ。"""
        y = self.predict(x)
        pred_label = np.argmax(y, axis=1)
        answer_label = np.argmax(t, axis=1)
        return np.sum(y == t) / float(x.shape[0])

    def numerical_gradient_layers(self, x, t):
        """それぞれの重み、バイアスの勾配を計算する。"""

        loss_W = lambda W: self.loss(x, t)

        grads = {}

        grads["W1"] = numerical_gradient(loss_W, self.params["W1"])
        grads["b1"] = numerical_gradient(loss_W, self.params["b1"])
        grads["W2"] = numerical_gradient(loss_W, self.params["W2"])
        grads["b2"] = numerical_gradient(loss_W, self.params["b2"])

        return grads

In [8]:
net = TwoLayerNet(input_size=784, hidden_size=50,
									output_size=10)

# それぞれのレイヤーのサイズを表示してみる
print(f"W1: {net.params['W1'].shape}")
print(f"b1: {net.params['b1'].shape}")
print(f"W2: {net.params['W2'].shape}")
print(f"b2: {net.params['b2'].shape}")

# 100枚分のミニバッチ（ダミーデータ）に対して推論を行ってみる
x = np.random.rand(100, 784)
y = net.predict(x)
print(f"\ny: {y.shape}")

# ダミーの正解データも作成し、勾配を求めてみる
t = np.random.rand(100, 10)
grads = net.numerical_gradient_layers(x, t)

W1: (784, 50)
b1: (50,)
W2: (50, 10)
b2: (10,)

y: (100, 10)
